In [195]:
#Импорт библиотек
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats as st
pd.options.mode.chained_assignment = None  # default='warn

In [196]:
data = pd.read_csv('megafon.csv') #Загрузка данных

In [197]:
data.head(3) #Cтруктура данных

,user_id,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming Download Throughput(Kbps),Video Streaming xKB Start Delay(ms),Web Page Download Throughput(Kbps),Web Average TCP RTT(ms)
0,1,5,NaN,775.48846,360.13,86.56,3.93,1859.15,2309,1007.82,83
1,2,5,4,861.96324,3023.54,411.18,1.27,667.47,2080,255.36,425
2,3,1,4,261.11860,790.96,34.20,1.79,1079.60,6367,535.85,485


**user_id** — идентификатор абонента;

**Q1 — ответ на первый вопрос** В ходе опроса компания «Мегафон» предложила своим клиентам оценить уровень удовлетворённости качеством связи по десятибалльной шкале (где 10 — это «отлично», а 1 — «ужасно»). Если клиент оценивал качество связи на 9 или 10 баллов, опрос заканчивался;

**Q2 - ответ на второй вопрос** ; Если клиент ставил оценку ниже 9, задавался второй вопрос — о причинах неудовлетворённости качеством связи с предоставленными пронумерованными вариантами ответа. Ответ можно было дать в свободном формате или перечислить номера ответов через запятую:
1. Недозвоны, обрывы при звонках
2. Время ожидания гудков при звонке
3. Плохое качество связи в зданиях, тц и т.д.
4. Медленный мобильный интернет
5. Медленная загрузка видео
6. Затрудняюсь ответить
7. Свой вариант

**Total Traffic(MB)** — объем трафика передачи данных, насколько активно абонент использует мобильный интернет;

**Downlink Throughput(Kbps)** — средняя скорость «к абоненту», считается по всему трафику передачи данных;

**Uplink Throughput(Kbps)**— средняя скорость «от абонента», считается по всему трафику передачи данных;

**Downlink TCP Retransmission Rate(%)** — частота переотправок пакетов «к абоненту», чем выше, тем хуже. Если в канале возникает ошибка, пакет переотправляется. Снижается полезная скорость;

**Video Streaming Download Throughput(Kbps)** — скорость загрузки потокового видео, чем выше, тем лучше — меньше прерываний и лучше качество картинки;

**Video Streaming xKB Start Delay(ms)** — задержка старта воспроизведения видео, cколько времени пройдёт между нажатием на кнопку Play и началом воспроизведения видео. Чем меньше это время, тем быстрее начинается воспроизведение;

**Web Page Download Throughput(Kbps)** — скорость загрузки web-страниц через браузер, чем выше, тем лучше;

**Web Average TCP RTT(ms)** — пинг при просмотре web-страниц, чем меньше, тем лучше — быстрее загружаются web-страницы.

Первый технический показатель представлен как сумма за период в одну неделю перед участием в опросе. Остальные технические показатели отображают среднее значение по данному признаку за период в одну неделю перед участием в опросе.

### Предобработка данных###

In [198]:
data.info() #Непустые значения в колонках

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3112 entries, 0 to 3111
Data columns (total 11 columns):
user_id                                      3112 non-null int64
Q1                                           3110 non-null object
Q2                                           1315 non-null object
Total Traffic(MB)                            3112 non-null float64
Downlink Throughput(Kbps)                    3112 non-null float64
Uplink Throughput(Kbps)                      3112 non-null float64
Downlink TCP Retransmission Rate(%)          3112 non-null float64
Video Streaming Download Throughput(Kbps)    3112 non-null float64
Video Streaming xKB Start Delay(ms)          3112 non-null int64
Web Page Download Throughput(Kbps)           3112 non-null float64
Web Average TCP RTT(ms)                      3112 non-null int64
dtypes: float64(6), int64(3), object(2)
memory usage: 267.6+ KB


**Работа с переменной Q1 - ответ на первый вопрос, возможные значения 1-10**

In [199]:
data['Q1'].value_counts() #Имеющиеся значения и их количество

10                                                                      846
1                                                                       532
3                                                                       325
8                                                                       291
9                                                                       238
5                                                                       234
7                                                                       200
2                                                                       168
4                                                                       123
6                                                                       101
0                                                                        10
Да                                                                        2
5, 6                                                                      2
Нет         

In [200]:
data['user_id'] = data['user_id'].astype(str)
data['Q1'] = data['Q1'].astype('str')
data['Q1'] = [elem if elem in ['1','2','3','4','5','6','7','8','9','10'] else np.NaN for elem in data['Q1']]
data.dropna(subset=['Q1'], inplace=True)
data['Q1'].value_counts()

10    846
1     532
3     325
8     291
9     238
5     234
7     200
2     168
4     123
6     101
Name: Q1, dtype: int64

**Разделим абонентов на группы и создадим метки группы для всех абонентов - A: оценка (9-10), B: оценка (5-8), C: оценка (1-4)**

In [201]:
data['Group'] = data['Q1']

In [202]:
data['Group'] = ['A' if elem in ['9','10'] else 'B' if elem in ['5','6','7','8'] else 'C' for elem in data['Group']]

In [203]:
data['Group'].value_counts() #Количество абонентов по группам

C    1148
A    1084
B     826
Name: Group, dtype: int64

In [204]:
#Изменим порядок колонок в датасете

In [205]:
cols = ['user_id', 'Group', 'Q1','Q2','Total Traffic(MB)','Downlink Throughput(Kbps)','Uplink Throughput(Kbps)', 
        'Video Streaming Download Throughput(Kbps)', 'Web Page Download Throughput(Kbps)',
        'Downlink TCP Retransmission Rate(%)', 'Video Streaming xKB Start Delay(ms)', 'Web Average TCP RTT(ms)']
data = data[cols]
data.head(1)

,user_id,Group,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
0,1,B,5,NaN,775.48846,360.13,86.56,1859.15,1007.82,3.93,2309,83


In [206]:
#Т.к. отсутствующие значения остались только в колонке Q2, проставим ответ 6 (затрудняюсь ответить) для пустых значений
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3058 entries, 0 to 3111
Data columns (total 12 columns):
user_id                                      3058 non-null object
Group                                        3058 non-null object
Q1                                           3058 non-null object
Q2                                           1315 non-null object
Total Traffic(MB)                            3058 non-null float64
Downlink Throughput(Kbps)                    3058 non-null float64
Uplink Throughput(Kbps)                      3058 non-null float64
Video Streaming Download Throughput(Kbps)    3058 non-null float64
Web Page Download Throughput(Kbps)           3058 non-null float64
Downlink TCP Retransmission Rate(%)          3058 non-null float64
Video Streaming xKB Start Delay(ms)          3058 non-null int64
Web Average TCP RTT(ms)                      3058 non-null int64
dtypes: float64(6), int64(2), object(4)
memory usage: 310.6+ KB


In [207]:
data[data['Group'] != 'A'] = data[data['Group'] != 'A'].fillna('6')

In [208]:
data[(data['Q1'] == '9') | (data['Q1'] == '10')][['Q2']].shape[0] #Проверим количество строк группы A

1084

In [209]:
data[(data['Q1'] == '9') | (data['Q1'] == '10')][['Q2']].isna().sum()[0] #Проверим пустые значения группы A на Q2 - совпадает

1084

In [210]:
data_Q1 = data.copy()

**Устранение выбросов**

Следуя эвристики, что выбросы находятся за пределами следующих интервалов: Q1–1.5 x IQR и Q3 + 1.5 x IQR, преобразуем данные.

Q1 - Первый квартиль, равен 25-ому процентилю;

Q3 - Третий квартиль, равен 75-ому процентилю;

IQR - число, которое показывает разброс средней половины (т.е. средние 50%) набора данных и помогает определить выбросы, разница между Q3 и Q1;

In [211]:
col = data.describe().columns #Колонки числовых атрибутов

In [212]:
Q1, Q3 = data_Q1[col].quantile(0.25), data_Q1[col].quantile(0.75)
IQR = Q3 - Q1
data_Q1_Out = data_Q1[~((data_Q1[col] < (Q1 - 1.5 * IQR)) |(data_Q1[col] > (Q3 + 1.5 * IQR))).any(axis=1)]
data = data_Q1_Out

**Работа с переменной Q2 - ответ на первый вопрос, возможные значения 1-7, или несколько ответов**

In [213]:
data.Q2.isna().sum()

700

In [214]:
data['Q2'].value_counts() 

6                   451
3                   135
4                   100
1                    98
1, 3                 80
3, 4                 65
1, 3, 4              43
7                    39
1, 2, 3              31
1, 4                 29
1, 3, 4, 5           28
3, 4, 5              27
1, 2, 3, 4, 5        19
1, 2                 16
4, 5                 16
1, 4, 5              13
1, 2, 3, 4           13
3, 5                  7
1, 2, 4               7
2, 3, 4               7
2, 3                  7
2                     6
1, 5                  5
5                     4
1, 3, 5               3
2, 3, 4, 5            3
2, 4                  3
1, 4, 7               3
1, 3, 4, 7            2
1, 3, 4, 5, 7         2
1, 2, 4, 5            2
1, 2, 5               2
2, 4, 5               2
1, 2, 34              1
3, 4, 7               1
0, 3                  1
0, 1, 7               1
1, 2, 7               1
0, 05, 2, 27, 7       1
1, 2, 3, 5            1
1, 2, 3, 4, 5, 6      1
0               

In [215]:
# Для наблюдений с несколькими вариантами, уберем пробелы, переведем в списки и разобьем на отдельные наблюдения.
# Значения остальных атрибутов для таких наблюдений остаются неизменными.

In [216]:
data['Q2'] = data['Q2'].str.replace(" ", "").str.split(',')
data = data.explode('Q2')

In [217]:
data['Q2'] = [elem if elem in ['1','2','3','4','5','6','7',np.NaN] else 'error' for elem in data['Q2']]
data = data[data['Q2'] != 'error']

In [218]:
data['Q2'].value_counts() #Финальное количество вариантов ответа на Q2

3    480
6    452
1    403
4    387
5    136
2    124
7     54
Name: Q2, dtype: int64

In [219]:
data[data['Group'] != 'A'] = data[data['Group'] != 'A'].fillna('6') #Заполним пропуски в Q2 для групп B и C нейтральным ответов
data = data.reset_index(drop=True)

data_Q2 = data.copy()

**Рассмотрим данные для минимальных и максимальных показателей каждого признака с соответствующими значениями других признаков**

In [220]:
datamin = pd.DataFrame(columns = data.columns)
datamax = pd.DataFrame(columns = data.columns) 

In [221]:
data_copy = data.copy() #Создадим копию данных и исключим ошибочные минимальные (нулевые) значения признаков
data_copy = data_copy[(data_copy['Video Streaming Download Throughput(Kbps)'] > 0) & (data_copy['Web Page Download Throughput(Kbps)'] > 0) & (data_copy['Web Average TCP RTT(ms)'] > 0)]

In [222]:
for column in data[col]:
    datamin = datamin.append(data_copy[data_copy[column]== data_copy[column].min()])
    datamax = datamax.append(data_copy[data_copy[column]== data_copy[column].max()])
    
datamin = datamin.drop_duplicates(subset='Total Traffic(MB)') #Исключим дубликаты из полученных датафреймов
datamax = datamax.drop_duplicates(subset='Total Traffic(MB)')

datamin.append(data_Q1_Out.describe().loc[['mean'], :], sort=False)

,user_id,Group,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
654,757,A,10,NaN,100.349410,2779.170000,252.600000,8952.330000,1408.670000,2.090000,1228,224
2284,2638,C,1,3,305.542840,33.500000,88.100000,13609.460000,1534.650000,0.480000,1495,88
403,459,A,10,NaN,302.325410,891.880000,10.060000,1082.740000,123.680000,3.250000,3441,191
1972,2264,C,3,1,213.918400,1367.530000,40.500000,120.510000,979.910000,4.020000,924,144
724,826,C,1,6,744.218970,1324.470000,74.540000,7351.950000,55.250000,1.070000,1510,133
2321,2674,A,10,NaN,757.931430,4828.010000,234.230000,6242.830000,1764.300000,0.080000,1983,193
2623,2986,A,10,NaN,840.896310,4095.370000,265.030000,4626.930000,2413.900000,0.080000,2478,58
1639,1903,A,10,NaN,251.488770,686.450000,36.550000,408.940000,2462.010000,0.280000,375,98
502,551,A,10,NaN,107.807190,2118.170000,356.350000,7615.330000,3096.080000,3.460000,2037,26
mean,NaN,NaN,NaN,NaN,416.991616,1787.639859,142.040151,5095.346779,1800.697542,1.484755,1775.22,140.243


In [223]:
datamax.append(data_Q1_Out.describe().loc[['mean'], :], sort=False)

,user_id,Group,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
519,576,B,7,6,885.376090,3230.860000,92.840000,5407.500000,2369.430000,1.520000,1935,50
1289,1489,B,5,4,311.683530,6042.710000,333.830000,10608.620000,1826.350000,0.140000,745,84
325,392,B,8,6,461.772290,3237.890000,418.830000,4836.110000,3025.960000,1.110000,1483,82
1733,1993,A,10,NaN,152.085790,5765.290000,84.030000,15421.780000,318.010000,1.300000,658,126
2231,2591,A,10,NaN,506.154750,5227.370000,346.340000,11123.220000,4757.220000,0.250000,952,80
2092,2410,C,2,6,391.826710,187.870000,13.750000,400.940000,339.800000,4.540000,1366,341
1426,1655,B,5,4,423.297500,1366.610000,69.120000,1896.580000,660.740000,1.800000,3838,330
54,85,C,1,6,683.193100,268.660000,64.920000,881.190000,378.310000,2.290000,2796,403
1880,2156,A,10,NaN,248.655760,652.390000,44.770000,2301.240000,1404.200000,3.320000,1510,403
mean,NaN,NaN,NaN,NaN,416.991616,1787.639859,142.040151,5095.346779,1800.697542,1.484755,1775.22,140.243


**Рассмотрим меры центральной тенденции - среднее и медиану для групп пользователей разделенных по ответу на первый вопрос:**

Группы - A: оценка (9-10), B: оценка (5-8), C: оценка (1-4)

In [224]:
Q1_mean_1 = data_Q1_Out.groupby(['Group'])[['user_id']].count()
Q1_mean_1.columns = ['user_id, count']
Q1_mean_2 = data_Q1_Out.groupby(['Group']).agg('mean').sort_values(by='Group').round(2)      
Q1_mean = pd.concat([Q1_mean_1, Q1_mean_2], axis=1) 

Q1_median_1 = data_Q1_Out.groupby(['Group'])[['user_id']].count()
Q1_median_1.columns = ['user_id, count']
Q1_median_2 = data_Q1_Out.groupby(['Group']).agg('median').sort_values(by='Group').round(2)        
Q1_median = pd.concat([Q1_median_1, Q1_median_2], axis=1) 

Q1_mean

,"user_id, count",Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
Group,,,,,,,,,
A,700,412.30,1876.49,145.13,5371.52,1879.51,1.38,1696.58,131.57
B,547,424.04,1842.24,144.26,5260.61,1824.31,1.49,1768.55,138.85
C,734,416.21,1662.22,137.44,4708.80,1707.93,1.58,1855.18,149.55


In [225]:
Q1_median

,"user_id, count",Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
Group,,,,,,,,,
A,700,365.00,1519.74,120.80,4821.22,1769.54,1.20,1572.5,105.5
B,547,389.35,1366.61,119.43,4623.44,1666.18,1.28,1655.0,114.0
C,734,377.32,1293.52,116.33,4109.82,1576.06,1.32,1694.0,127.5


Показатели: Total Traffic(MB) - из опрошенных мобильный интернет активнее всего используется пользователями группы 'B';

Downlink Throughput(Kbps),Uplink Throughput(Kbps), Video Streaming Download Throughput(Kbps), Web Page Download Throughput(Kbps) - наибольшие показатели скорости загрузки у пользователей группы 'A', наименьшие у группы 'C'; 

Downlink TCP Retransmission Rate(%)(частота переотправок пакетов «к абоненту»),Video Streaming xKB Start Delay(ms)(задержка старта воспроизведения видео),Web Average TCP RTT(ms)(пинг при просмотре web-страниц) - наименьшие (наилучшие) показатели у пользователей группы 'A', наибольшие (наихудшие) - у группы 'C';

**Рассмотрим меры центральной тенденции - среднее и медиану для групп пользователей разделенных по ответу на второй вопрос:**

1.Недозвоны, обрывы при звонках  
2.Время ожидания гудков при звонке  
3.Плохое качество связи в зданиях, тц и т.д.  
4.Медленный мобильный интернет  
5.Медленная загрузка видео  
6.Затрудняюсь ответить  
7.Свой вариант  

In [226]:
Q2_mean_1 = data_Q2.groupby(['Q2'])[['user_id']].count()
Q2_mean_1.columns = ['user_id, count']
Q2_mean_2 = data_Q2.groupby(['Q2']).agg('mean').sort_values(by='Q2').round(2)       
Q2_mean = pd.concat([Q2_mean_1, Q2_mean_2], axis=1) 

Q2_median_1 = data_Q2.groupby(['Q2'])[['user_id']].count()
Q2_median_1.columns = ['user_id, count']
Q2_median_2 = data_Q2.groupby(['Q2']).agg('median').sort_values(by='Q2').round(2)       
Q2_median = pd.concat([Q2_median_1, Q2_median_2], axis=1) 

Q2_mean

,"user_id, count",Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
Q2,,,,,,,,,
1,403,410.06,1787.28,145.94,5045.35,1756.82,1.59,1817.25,145.68
2,124,395.59,1654.47,140.41,4793.18,1770.88,1.56,1847.86,153.37
3,480,420.52,1728.35,141.19,4817.77,1769.04,1.62,1809.03,142.59
4,387,440.76,1661.62,135.53,4482.69,1648.09,1.59,1902.98,154.15
5,136,434.34,1447.95,131.38,3919.86,1613.10,1.58,1929.76,157.15
6,452,412.84,1684.16,142.23,5047.90,1769.47,1.50,1796.79,146.30
7,54,406.87,1753.45,138.13,4808.98,1803.71,1.52,1817.15,131.33


In [227]:
Q2_median

,"user_id, count",Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
Q2,,,,,,,,,
1,403,370.92,1361.30,119.45,4576.39,1589.45,1.37,1662.0,125.0
2,124,375.97,1095.54,116.10,4050.38,1628.90,1.32,1709.5,131.5
3,480,379.86,1314.74,121.15,4252.07,1644.94,1.34,1664.5,117.0
4,387,415.86,1155.32,118.53,3915.89,1498.28,1.36,1743.0,133.0
5,136,413.47,1108.19,104.55,3287.10,1430.19,1.33,1743.5,135.5
6,452,366.52,1265.22,116.13,4349.46,1641.42,1.27,1671.0,122.0
7,54,352.47,1543.91,124.46,4223.85,1513.76,1.39,1700.5,101.0


На первый взгляд - видимых различий не наблюдается, более детальные различия и гипотезы между данными группами далее будут проверены в ходе исследования

**Оценка доверительных интервалов для оценки долей пользователей, которые полностью довольны качеством связи (группа A), и для пользователей, которые неудовлетворёны качеством связи (группы В и С)**

In [228]:
data_Q1[['user_id']].duplicated().sum()

0

In [229]:
df1 = data_Q1.copy()
df1['Group'] = ['Satisfied' if elem == 'A' else 'Dissatisfied' for elem in df1['Group']]
df1 = df1.groupby(['Group'])[['user_id']].count()
px.bar(df1, color=df1.index, title ='Количество пользователей в группах удовлетворенности услугами')

Расчет доверительного интервала для доли пользователей полностью удовлетворенными качеством связи (Ответ 9,10 на первый вопрос)
генеральной совокупности пользователей компании 'Мегафон'

In [230]:
n = data_Q1.shape[0] #размер выборки
n

3058

In [231]:
p = data_Q1[data_Q1['Group'] == 'A'].shape[0] / n

se = np.sqrt(p * (1 - p) / n)

np.round(st.norm.interval(0.95, loc = p, scale = se), 4)

array([0.3375, 0.3714])

Расчет доверительного интервала для доли пользователей  неудовлетворенными качеством связи (Ответ 1-8 на первый вопрос)
генеральной совокупности пользователей компании 'Мегафон'

In [232]:
n = data_Q1.shape[0]

p = data_Q1[data_Q1['Group'] != 'A'].shape[0] / n

se = np.sqrt(p * (1 - p) / n)

np.round(st.norm.interval(0.95, loc = p, scale = se), 4)

array([0.6286, 0.6625])

**С вероятностью в 95% доля пользователей полностью удовлетворенными качеством связи компании 'Мегафон' лежит в промежутке (0.3375; 0.3714), доля пользоваталей неудовлетворенными качеством связи лежит в промежутке (0.6286; 0.6625)**

**Рассмотрим показатели скорости загрузки потокового видео и задержки старта воспроизведения для пользователей с медленной загрузкой видео**

In [233]:
data_Q2.groupby(['Q2'])[['Downlink Throughput(Kbps)','Video Streaming Download Throughput(Kbps)',
                             'Video Streaming xKB Start Delay(ms)']] \
        .agg(['count' ,'mean', 'median']).sort_values(by='Q2').iloc[:, [0,1,2,4,5,7,8]] \
        .style.apply(lambda x: ['background: lightblue' if x.name == '5' else '' for i in x], axis=1)

In [234]:
data_Q1_Out.groupby(['Group'])[['Downlink Throughput(Kbps)','Video Streaming Download Throughput(Kbps)', 
                                                             'Video Streaming xKB Start Delay(ms)']] \
                                        .agg(['count' ,'mean', 'median']).iloc[:, [0,1,2,4,5,7,8]].round(2) \
                                        .style.apply(lambda x: ['background: orange' if x.name == 'A' else '' for i in x], axis=1)

In [235]:
#Наблюдения в выборках группы A и ответивших на второй вопрос независимы
data_Q2['Video Streaming Download Throughput(Kbps)'][(data_Q2['Q1'] == 'A') | (data_Q2['Q2'] == '5')].duplicated().sum()

0

In [236]:
fig = px.scatter(data_Q1_Out , x = 'Video Streaming xKB Start Delay(ms)', y = 'Video Streaming Download Throughput(Kbps)',
                 color = 'Group', title='Зависимость скорости загрузки видео от задержки начала воспроизведения')
fig.show()

In [237]:
fig = px.scatter(data_Q1_Out , x = 'Downlink Throughput(Kbps)', y = 'Video Streaming Download Throughput(Kbps)',
                 color = 'Group', title='Зависимость скорости загрузки видео от скорости "к абоненту"')
fig.show()

In [238]:
fig = px.histogram(data_Q1_Out, x='Video Streaming Download Throughput(Kbps)', title ='Распределение скорости загрузки потокового видео')
fig.show()

Рассмотрение независимых наблюдений в двух группах одного типа

In [239]:
mass = data_Q2[(data_Q2['Q2'] == '3') | (data_Q2['Q2'] == '4')]
indexes = pd.DataFrame(mass['user_id'].value_counts())
indexes = indexes[indexes['user_id'] == 2].index
mass['user_id'] = [np.NaN if elem in indexes else elem for elem in mass['user_id']]
mass = mass.dropna()
mass

,user_id,Group,Q1,Q2,Total Traffic(MB),Downlink Throughput(Kbps),Uplink Throughput(Kbps),Video Streaming Download Throughput(Kbps),Web Page Download Throughput(Kbps),Downlink TCP Retransmission Rate(%),Video Streaming xKB Start Delay(ms),Web Average TCP RTT(ms)
1,4,B,8,3,179.18564,2590.97,325.88,7053.81,1221.02,0.80,3218,51
13,11,C,3,3,526.08652,535.54,208.67,2621.14,2376.50,1.46,1479,88
16,19,B,7,3,811.55618,460.32,65.20,1583.27,1587.16,1.50,1340,57
31,50,C,2,3,453.12793,3325.86,167.74,11702.86,2455.82,0.59,1212,40
35,53,B,6,3,168.11047,2031.20,164.31,5234.92,1665.56,0.71,1838,95
...,...,...,...,...,...,...,...,...,...,...,...,...
2705,3095,C,3,3,566.73519,3477.18,124.84,5427.74,2410.91,0.76,1188,145
2718,3101,C,2,4,248.99905,2347.42,309.08,5001.39,1837.79,1.64,1394,83
2719,3102,B,8,4,189.14150,2432.18,72.80,2152.91,1410.52,0.70,2719,289
2727,3108,C,3,4,519.96475,1045.70,44.61,4523.66,1044.66,0.47,1468,304


In [240]:
mass.Q2.value_counts()

3    268
4    175
Name: Q2, dtype: int64